In [1]:
import numpy as np
import tqdm
import pandas as pd
import pydeck as pdk

# Process data

In [2]:
df = pd.read_parquet('data/geolife_trajectories_1_3.parquet')
print(df.shape[0])
df.head()

24876978


,lat,lon,datetime,altitude_meters,tripId,userId
0,22.816448,108.332067,2009-02-04 09:57:32,0.0,023_20090204095732,023
1,22.815663,108.332210,2009-02-04 09:57:37,0.0,023_20090204095732,023
2,22.815722,108.332180,2009-02-04 09:57:42,0.0,023_20090204095732,023
3,22.815742,108.332167,2009-02-04 09:57:47,0.0,023_20090204095732,023
4,22.815738,108.332145,2009-02-04 09:57:52,0.0,023_20090204095732,023


# Pydeck - deck.gl python binding

The pydeck library is a set of Python bindings for making spatial visualizations with deck.gl, optimized for a Jupyter Notebook environment.

Lots of examples here: https://github.com/uber/deck.gl/tree/master/bindings/pydeck/examples


# Load the data into pydeck

In [7]:
# Make a smaller dataframe due to memory restraints
df_small  = df.head(500_000)

In [13]:
layer_hex = pdk.Layer(
    'HexagonLayer',
    df_small,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1)

layer_scatter = pdk.Layer(
    'ScatterplotLayer',
    df_small,
    get_radius=500,
    get_fill_color='color',
    get_position='position')

In [11]:
# Set the viewport location
viewport = pdk.data_utils.compute_view(points=df_small[['lon', 'lat']], view_proportion=0.7)

In [14]:
# Render
r = pdk.Deck(layers=[layer_hex, layer_scatter], initial_view_state=viewport)
r.to_html('data/pydeck.html')

'/home/tjansson/code/geospatial/data/pydeck.html'